In [132]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [44]:
dfamz = pd.read_csv('E:/KrishM/PythonPrjs/PyDatasets/sentiments/amazonrev.txt', delimiter='\t', header=None)

In [45]:
dfamz

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [46]:
dfamz.columns = ['Reviews', 'Class']

In [47]:
dfamz

,Reviews,Class
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [48]:
dfimd = pd.read_csv('E:/KrishM/PythonPrjs/PyDatasets/sentiments/imdbrev.txt', delimiter='\t', header=None)

In [49]:
dfimd

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


In [50]:
dfimd.columns = ['Reviews', 'Class']

In [51]:
dfylp = pd.read_csv('E:/KrishM/PythonPrjs/PyDatasets/sentiments/yelprev.txt', delimiter='\t', header=None)

In [52]:
dfylp

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [53]:
dfylp.columns = ['Reviews', 'Class']

In [54]:
dfylp

,Reviews,Class
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [57]:
dfaiy = pd.concat([dfamz, dfimd, dfylp], ignore_index=True)

In [58]:
dfaiy

,Reviews,Class
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0


In [67]:
dfaiy.iloc[2747]

Reviews    Then, as if I hadn't wasted enough of my life ...
Class                                                      0
Name: 2747, dtype: object

In [115]:
dfaiy['Reviews'].head(25)

0     So there is no way for me to plug it in here i...
1                           Good case, Excellent value.
2                                Great for the jawbone.
3     Tied to charger for conversations lasting more...
4                                     The mic is great.
5     I have to jiggle the plug to get it to line up...
6     If you have several dozen or several hundred c...
7           If you are Razr owner...you must have this!
8                   Needless to say, I wasted my money.
9                      What a waste of money and time!.
10                      And the sound quality is great.
11    He was very impressed when going from the orig...
12    If the two were seperated by a mere 5+ ft I st...
13                             Very good quality though
14    The design is very odd, as the ear "clip" is n...
15    Highly recommend for any one who has a blue to...
16                  I advise EVERYONE DO NOT BE FOOLED!
17                                     So Far So

In [104]:
dfvaiyval = dfaiy['Reviews'].values

In [105]:
dfvaiyval

array(['So there is no way for me to plug it in here in the US unless I go by a converter.',
       'Good case, Excellent value.', 'Great for the jawbone.', ...,
       'Overall I was not impressed and would not go back.',
       "The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.",
       "Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing out the time it took to bring the check."],
      dtype=object)

In [106]:
dfvaiyval.shape

(2748,)

In [107]:
type(dfvaiyval)

numpy.ndarray

In [110]:
dfvaiyval[77]

'This is a great little item.'

In [111]:
type(dfvaiyval[77])

str

In [112]:
dfvaiyval.tolist()

['So there is no way for me to plug it in here in the US unless I go by a converter.',
 'Good case, Excellent value.',
 'Great for the jawbone.',
 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!',
 'The mic is great.',
 'I have to jiggle the plug to get it to line up right to get decent volume.',
 'If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.',
 'If you are Razr owner...you must have this!',
 'Needless to say, I wasted my money.',
 'What a waste of money and time!.',
 'And the sound quality is great.',
 'He was very impressed when going from the original battery to the extended battery.',
 'If the two were seperated by a mere 5+ ft I started to notice excessive static and garbled sound from the headset.',
 'Very good quality though',
 'The design is very odd, as the ear "clip" is not very comfortable at all.',
 'Highly recommend for any one who has a blue tooth phone.',
 'I advise EVERYO

In [116]:
dfaiy

,Reviews,Class
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0


In [117]:
dfaiy['Reviews']

0       So there is no way for me to plug it in here i...
1                             Good case, Excellent value.
2                                  Great for the jawbone.
3       Tied to charger for conversations lasting more...
4                                       The mic is great.
                              ...                        
2743    I think food should have flavor and texture an...
2744                             Appetite instantly gone.
2745    Overall I was not impressed and would not go b...
2746    The whole experience was underwhelming, and I ...
2747    Then, as if I hadn't wasted enough of my life ...
Name: Reviews, Length: 2748, dtype: object

In [118]:
type(dfaiy['Reviews'])

pandas.core.series.Series

In [133]:
def clean_text(df):
    all_reviews = []
    lines = dfaiy['Reviews'].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r'[,.\"!@#$%^&*(){}?/;~:<>+=-]', "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('','', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        stop_words.discard('not')
        PS = PorterStemmer()
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

all_reviews = clean_text(dfaiy)
all_reviews[0:20]    
                      
                      

['way plug us unless go convert',
 'good case excel valu',
 'great jawbon',
 'tie charger convers last minutesmajor problem',
 'mic great',
 'jiggl plug get line right get decent volum',
 'sever dozen sever hundr contact imagin fun send one one',
 'razr ownery must',
 'needless say wast money',
 'wast money time',
 'sound qualiti great',
 'impress go origin batteri extend batteri',
 'two seper mere ft start notic excess static garbl sound headset',
 'good qualiti though',
 'design odd ear clip not comfort',
 'highli recommend one blue tooth phone',
 'advis everyon not fool',
 'far good',
 'work great',
 'click place way make wonder long mechan would last']

In [134]:
all_reviews

['way plug us unless go convert',
 'good case excel valu',
 'great jawbon',
 'tie charger convers last minutesmajor problem',
 'mic great',
 'jiggl plug get line right get decent volum',
 'sever dozen sever hundr contact imagin fun send one one',
 'razr ownery must',
 'needless say wast money',
 'wast money time',
 'sound qualiti great',
 'impress go origin batteri extend batteri',
 'two seper mere ft start notic excess static garbl sound headset',
 'good qualiti though',
 'design odd ear clip not comfort',
 'highli recommend one blue tooth phone',
 'advis everyon not fool',
 'far good',
 'work great',
 'click place way make wonder long mechan would last',
 'went motorola websit follow direct could not get pair',
 'bought use kindl fire absolut love',
 'commerci mislead',
 'yet run new batteri two bar three day without charg',
 'bought mother problem batteri',
 'great pocket pc phone combin',
 'own phone month say best mobil phone',
 'nt think instruct provid help',
 'peopl couldnt hea